In [42]:
# !pip install hsclient
targets = ["beta.hydroshare.org", "www.hydroshare.org"]
clients = {}
filename = "local-build-discover.sh"
diffs= {}

In [43]:
from hsclient import HydroShare
for target in targets:
    hs = HydroShare(host=target, port=443, protocol='https')
    hs.sign_in()
    hs.resources = []
    diffs[target] = []
    clients[target] = hs

In [58]:
from time import time
from functools import wraps
import numpy as np
import scipy
  
def timer_func(func): 
    @wraps(func)
    def wrap_func(*args, **kwargs): 
        t1 = time() 
        result = func(*args, **kwargs)
        t2 = time()
        target = kwargs['target']
        diff = t2-t1
        diffs[target].append(diff)
        print(f'Function {func.__name__!r} executed in {(diff):.4f}s') 
        return result 
    return wrap_func


def create_res(hs, time=True, target=None):

    @timer_func
    def create_res_time(hs=None, target=None):
        resource = hs.create()
        hs.resources.append(resource)
    
    if time:
        create_res_time(hs=hs, target=target)
    else:
        resource = hs.create()
        hs.resources.append(resource)

@timer_func
def delete_resources(hs=None, target=None):
    for res in hs.resources:
        res.delete()

@timer_func
def download_all_resources(hs=None, target=None):
    for res in hs.resources:
        try:
            res.download()
        except Exception as e:
            print(e)

def download_single_resource(hs=None, target=None):
    create_res(hs, False)
    res = hs.resources[-1]
    res.file_upload(filename)

    @timer_func
    def download_single_resource_inner(hs=res, target=None):
        res.download()

    download_single_resource_inner(hs=res, target=target)

def delete_single_resource(hs=None, target=None):
    create_res(hs, False)
    res = hs.resources[-1]
    res.file_upload(filename)

    @timer_func
    def delete_single_resource_inner(hs=res, target=None):
        res.delete()

    delete_single_resource_inner(hs=res, target=target)

def upload_file(hs, target=None):
    create_res(hs, False)
    res = hs.resources[-1]

    @timer_func
    def upload_file_inner(hs=res, target=None):
        res.file_upload(filename)

    upload_file_inner(hs=res, target=target)


def create_dir_upload_file(hs=None, target=None):
    create_res(hs, False)
    res = hs.resources[-1]

    @timer_func
    def create_dir_upload_file_inner(hs=res, target=None):
        res.folder_create('New_Folder')
        res.file_upload(filename, destination_path='New_Folder')
    create_dir_upload_file_inner(hs=res, target=target)

def upload_and_movefile(hs=None, target=None):
    create_res(hs, False)
    res = hs.resources[-1]
    
    @timer_func
    def upload_and_movefile_inner(hs=res, target=None):
        res.folder_create('New_Folder')
        res.file_upload(filename, destination_path='New_Folder')
        file = res.file(path=f"New_Folder/{filename}]")
        res.file_rename(file, filename)
    upload_and_movefile_inner(hs=res, target=target)

def download_single_file(hs=None, target=None):
    create_res(hs, False)
    res = hs.resources[-1]
    res.file_upload(filename)

    @timer_func
    def download_single_file_inner(hs=res, target=target):
        file = res.file(path=filename)
        res.file_download(file)
    download_single_file_inner(hs=res, target=target)

def delete_single_file(hs=None, target=None):
    create_res(hs, False)
    res = hs.resources[-1]
    res.file_upload(filename)

    @timer_func
    def delete_single_file_inner(hs=res, target=None):
        file = res.file(path=filename)
        res.file_delete(file)
    delete_single_file_inner(hs=res, target=target)
        

def run_comparisons(functions_to_run, runs=1):
    for funct in functions_to_run:
        print(f"********* Start {funct.__name__!r} *********")
        for i in range(runs):
            for target in targets:
                client = clients[target]
                print(f"Starting run {i} of {funct.__name__} on {target}...")
                funct(hs=client, target=target)
                print(diffs[target])
        for target in targets:
            client = clients[target]
            print(f"Average time {funct.__name__!r} on {target}: {sum(diffs[target])/len(diffs[target])}")
            print(f"Devation for {funct.__name__!r} on {target}: {np.std(diffs[target])}")
        # https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html
        result = scipy.stats.ttest_ind(diffs[targets[0]], diffs[targets[1]])
        print(result)
        print("\n\n")
        for target in targets:
            diffs[target].clear()

def clear_resources():
    for target in targets:
        client = clients[target]
        client.resources = []
        diffs[target].clear()
  

In [46]:
clear_resources()
run_comparisons([download_single_resource, upload_file, create_dir_upload_file, create_res ], runs=3)
run_comparisons([download_all_resources, delete_resources], runs=1)

********* Start 'download_single_resource' *********
Starting run 0 of download_single_resource on beta.hydroshare.org...
Function 'download_single_resource_inner' executed in 37.9298s
[37.92979097366333]
Starting run 0 of download_single_resource on www.hydroshare.org...
Function 'download_single_resource_inner' executed in 19.7112s
[19.7112238407135]
Starting run 1 of download_single_resource on beta.hydroshare.org...
Function 'download_single_resource_inner' executed in 34.1435s
[37.92979097366333, 34.1435010433197]
Starting run 1 of download_single_resource on www.hydroshare.org...
Function 'download_single_resource_inner' executed in 19.8524s
[19.7112238407135, 19.85241198539734]
Starting run 2 of download_single_resource on beta.hydroshare.org...
Function 'download_single_resource_inner' executed in 39.8796s
[37.92979097366333, 34.1435010433197, 39.87956357002258]
Starting run 2 of download_single_resource on www.hydroshare.org...
Function 'download_single_resource_inner' execute

In [52]:
# Let's see if we can bump N and get some significance...
clear_resources()
run_comparisons([upload_file], runs=10)

********* Start 'upload_file' *********
Starting run 0 of upload_file on beta.hydroshare.org...
Function 'upload_file_inner' executed in 2.8627s
[2.8627121448516846]
Starting run 0 of upload_file on www.hydroshare.org...
Function 'upload_file_inner' executed in 2.9627s
[2.9626917839050293]
Starting run 1 of upload_file on beta.hydroshare.org...
Function 'upload_file_inner' executed in 2.8628s
[2.8627121448516846, 2.862791061401367]
Starting run 1 of upload_file on www.hydroshare.org...
Function 'upload_file_inner' executed in 2.9640s
[2.9626917839050293, 2.9640250205993652]
Starting run 2 of upload_file on beta.hydroshare.org...
Function 'upload_file_inner' executed in 2.8802s
[2.8627121448516846, 2.862791061401367, 2.8802077770233154]
Starting run 2 of upload_file on www.hydroshare.org...
Function 'upload_file_inner' executed in 2.9651s
[2.9626917839050293, 2.9640250205993652, 2.965050220489502]
Starting run 3 of upload_file on beta.hydroshare.org...
Function 'upload_file_inner' execu

Exception: Failed GET https://beta.hydroshare.org:443/resource/29d9e616898042dfb1bf4f64222534a5/data/resourcemap.xml/, status_code 404, message b'{"detail":"No resource was found for resource id:29d9e616898042dfb1bf4f64222534a5"}'

In [ ]:
clear_resources()
run_comparisons([create_res, download_all_resources], runs=5)

********* Start 'create_res' *********
Starting run 0 of create_res on beta.hydroshare.org...
Function 'create_res_time' executed in 29.6942s
[29.694232940673828]
Starting run 0 of create_res on www.hydroshare.org...
Function 'create_res_time' executed in 14.5412s
[14.541159868240356]
Starting run 1 of create_res on beta.hydroshare.org...
Function 'create_res_time' executed in 28.9799s
[29.694232940673828, 28.97987699508667]
Starting run 1 of create_res on www.hydroshare.org...
Function 'create_res_time' executed in 14.0283s
[14.541159868240356, 14.028294801712036]
Starting run 2 of create_res on beta.hydroshare.org...
Function 'create_res_time' executed in 26.3164s
[29.694232940673828, 28.97987699508667, 26.316428899765015]
Starting run 2 of create_res on www.hydroshare.org...
Function 'create_res_time' executed in 14.4391s
[14.541159868240356, 14.028294801712036, 14.439146041870117]
Starting run 3 of create_res on beta.hydroshare.org...
Function 'create_res_time' executed in 31.3342s

KeyboardInterrupt: 

In [62]:
clear_resources()
run_comparisons([delete_single_file], runs=3)

********* Start 'delete_single_file' *********
Starting run 0 of delete_single_file on beta.hydroshare.org...
Function 'delete_single_file_inner' executed in 29.7061s
[29.706119298934937]
Starting run 0 of delete_single_file on www.hydroshare.org...
Function 'delete_single_file_inner' executed in 15.5633s
[15.56330394744873]
Starting run 1 of delete_single_file on beta.hydroshare.org...
Function 'delete_single_file_inner' executed in 23.9646s
[29.706119298934937, 23.964612007141113]
Starting run 1 of delete_single_file on www.hydroshare.org...
Function 'delete_single_file_inner' executed in 15.2583s
[15.56330394744873, 15.258327960968018]
Starting run 2 of delete_single_file on beta.hydroshare.org...
Function 'delete_single_file_inner' executed in 24.8875s
[29.706119298934937, 23.964612007141113, 24.887469053268433]
Starting run 2 of delete_single_file on www.hydroshare.org...
Function 'delete_single_file_inner' executed in 15.4993s
[15.56330394744873, 15.258327960968018, 15.4993269443

In [64]:
clear_resources()
run_comparisons([delete_single_resource], runs=5)

********* Start 'delete_single_resource' *********
Starting run 0 of delete_single_resource on beta.hydroshare.org...
Function 'delete_single_resource_inner' executed in 31.3772s
[31.377173900604248]
Starting run 0 of delete_single_resource on www.hydroshare.org...
Function 'delete_single_resource_inner' executed in 18.7971s
[18.79710602760315]
Starting run 1 of delete_single_resource on beta.hydroshare.org...
Function 'delete_single_resource_inner' executed in 40.7536s
[31.377173900604248, 40.75357985496521]
Starting run 1 of delete_single_resource on www.hydroshare.org...
Function 'delete_single_resource_inner' executed in 18.4441s
[18.79710602760315, 18.444146871566772]
Starting run 2 of delete_single_resource on beta.hydroshare.org...
Function 'delete_single_resource_inner' executed in 36.8906s
[31.377173900604248, 40.75357985496521, 36.89060711860657]
Starting run 2 of delete_single_resource on www.hydroshare.org...
Function 'delete_single_resource_inner' executed in 18.3291s
[18.

In [65]:
run_comparisons([download_all_resources], runs=5)

********* Start 'download_all_resources' *********
Starting run 0 of download_all_resources on beta.hydroshare.org...
Failed GET https://beta.hydroshare.org:443/resource/260c89f293ae43039829bc6044a86991/data/resourcemap.xml/, status_code 404, message b'{"detail":"No resource was found for resource id:260c89f293ae43039829bc6044a86991"}'
Failed GET https://beta.hydroshare.org:443/resource/3ab86d344f9f46babfb1397347bc9aa4/data/resourcemap.xml/, status_code 404, message b'{"detail":"No resource was found for resource id:3ab86d344f9f46babfb1397347bc9aa4"}'
Failed GET https://beta.hydroshare.org:443/resource/46fc79db9a344665bc4fb48cf6f33561/data/resourcemap.xml/, status_code 404, message b'{"detail":"No resource was found for resource id:46fc79db9a344665bc4fb48cf6f33561"}'
Failed GET https://beta.hydroshare.org:443/resource/76cca452f523463292c9a3476756965f/data/resourcemap.xml/, status_code 404, message b'{"detail":"No resource was found for resource id:76cca452f523463292c9a3476756965f"}'
Fa

In [66]:
# TODO: these ones still seem not to work...
clear_resources()
run_comparisons([upload_and_movefile, download_single_file ], runs=2)

********* Start 'upload_and_movefile' *********
Starting run 0 of upload_and_movefile on beta.hydroshare.org...


Exception: Failed POST https://beta.hydroshare.org:443/hsapi/resource/d62d2796cb6c432088ce3c810763e9cb/functions/move-or-rename/, status_code 400, message b'{"error": "[\'A value for path is missing\']"}'